In [1]:
import numpy as np
import tensorflow as tf
import h5py
import random
slim=tf.contrib.slim

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
f=h5py.File('D:/data/wind/fitt.h5','r')
data_x=f['x']
data_y=f['y']
data_x[:2]

array([[18.08,  2.49],
       [18.14,  2.48]])

# 数据准备

In [3]:
#读取数据
f=h5py.File('D:/data/wind/fitt.h5','r')
data_x=f['x']
data_y=f['y']


#分配训练测试集
num=list(range(data_x.shape[0]))
num1=random.sample(num,1000)
num2=set(num)-set(num1)
num2=list(num2)
num1.sort()


x_train=data_x[num2]
y_train=data_y[num2]

x_test=data_x[num1]
y_test=data_y[num1]
#生成批次
train_queue = tf.train.slice_input_producer([x_train,y_train],shuffle=None)
val_queue = tf.train.slice_input_producer([x_test,y_test],shuffle=None)
batch_xt,batch_yt=tf.train.shuffle_batch(train_queue,batch_size=16,capacity=500,min_after_dequeue=150)
batch_xv,batch_yv=tf.train.shuffle_batch(val_queue,batch_size=16,capacity=500,min_after_dequeue=150)

# 搭建模型

### 超参数设置

In [4]:
# Optmizer: 
learning_rate = 0.007  # Small lr helps not to diverge during training. 
nb_iters = 10000  # How many times we perform a training step (therefore how many times we show a batch). 
lr_decay = 0.92  # default: 0.9 . Simulated annealing.
momentum = 0.5  # default: 0.0 . Momentum technique in weights update
lambda_l2_reg = 0.003  # L2 regularization of weights - avoids overfitting

### 搭建神经网络

In [5]:
x=tf.placeholder(tf.float32,[None,2])
y=tf.placeholder(tf.float32,[None,1])
with slim.arg_scope([slim.layers.fully_connected],
                    weights_initializer=slim.xavier_initializer(),
                    activation_fn=tf.nn.relu):
    net=slim.fully_connected(x,4,scope='fc1')
#     net=slim.fully_connected(net,8,scope='fc2')
    net=slim.fully_connected(net,4,scope='fc3')
    output=slim.fully_connected(net,1,activation_fn=None,scope='fc4')

### 搭建损失函数 

In [6]:
# Training loss and optimizer

with tf.variable_scope('Loss'):
    # L2 loss
   
    output_loss = tf.reduce_mean(tf.nn.l2_loss(output - y))
        
    # L2 regularization (to avoid overfitting and to have a  better generalization capacity)
    reg_loss = 0
    for tf_var in tf.trainable_variables():
        if not ("Bias" in tf_var.name or "Output_" in tf_var.name):
            reg_loss += tf.reduce_mean(tf.nn.l2_loss(tf_var))
            
    loss = output_loss + lambda_l2_reg * reg_loss
    tf.summary.scalar('loss',loss)

with tf.variable_scope('acc'):
    acc = tf.reduce_mean(tf.nn.l2_loss(y-output))/tf.reduce_mean(tf.nn.l2_loss(y-0))
    
    acc=1-acc
    tf.summary.scalar('acc',acc)

with tf.variable_scope('Optimizer'):
    optimizer = tf.train.RMSPropOptimizer(learning_rate, decay=lr_decay, momentum=momentum)
    train_op = optimizer.minimize(loss)
    merged_summary = tf.summary.merge_all()

### 训练模型

In [7]:
min_loss=10000
sess=tf.Session()
train_writer=tf.summary.FileWriter('C:/graph/fitted/train/',sess.graph)
test_writer = tf.summary.FileWriter('C:/graph/fitted/test/', sess.graph)
saver=tf.train.Saver(max_to_keep=3)

init_op = tf.global_variables_initializer()
sess.run(init_op)
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess, coord)

for training_itr in range(50000):
    x1, y1 = sess.run([batch_xt,batch_yt])
    feed_dict1 = {x:x1,y:y1}
    _, loss1,acc1,summaries1 = sess.run([train_op, loss,acc,merged_summary], feed_dict1)

    train_writer.add_summary(summaries1, training_itr)
    if training_itr %5==0:
#             saver.save(sess=sess, save_path='model/hand_landmark_v6.1_model/model.ckpt',global_step=(global_step + 1))
        mean_val_loss = 0

        x2,y2=sess.run([batch_xv,batch_yv])
        feed_dict2 = {x:x2,y:y2}
        loss2,acc2,summaries2 = sess.run([loss,acc,merged_summary], feed_dict2)

        print('step: {}'.format(training_itr))
        print('train: loss: {} acc: {}  val: loss: {} acc: {}'.format(loss1,acc1,loss2,acc2))
        test_writer.add_summary(summaries2, training_itr)
        if loss1 < min_loss:
            min_loss=loss1
            saver.save(sess=sess, save_path='D:/model/fitted/model.ckpt',global_step=(training_itr + 1))
sess.close()
coord.request_stop()
    






step: 0
train: loss: 3380332.5 acc: 0.0  val: loss: 8976114.0 acc: 3.4749507904052734e-05
step: 5
train: loss: 7107475.5 acc: 0.0002345442771911621  val: loss: 5024643.0 acc: 0.0002886652946472168
step: 10
train: loss: 3517692.0 acc: 0.0005472898483276367  val: loss: 4183223.0 acc: 0.0005928277969360352
step: 15
train: loss: 3611249.5 acc: 0.0007811188697814941  val: loss: 3025991.5 acc: 0.0009829401969909668
step: 20
train: loss: 6083501.0 acc: 0.0008274316787719727  val: loss: 5969022.0 acc: 0.0006934404373168945
step: 25
train: loss: 6104327.0 acc: 0.0009404420852661133  val: loss: 7096881.0 acc: 0.0007951855659484863
step: 30
train: loss: 5970197.5 acc: 0.0009676814079284668  val: loss: 6694899.5 acc: 0.0008764863014221191
step: 35
train: loss: 5052377.5 acc: 0.0012707710266113281  val: loss: 8711819.0 acc: 0.0009052157402038574
step: 40
train: loss: 12240496.0 acc: 0.0008459687232971191  val: loss: 8622820.0 acc: 0.0009105801582336426
step: 45
train: loss: 13268662.0 acc: 0.001018

step: 450
train: loss: 1079340.5 acc: 0.8640819787979126  val: loss: 3351078.0 acc: 0.5082187652587891
step: 455
train: loss: 1473493.75 acc: 0.5906012058258057  val: loss: 2305908.0 acc: 0.5285784602165222
step: 460
train: loss: 1292206.375 acc: 0.4157140254974365  val: loss: 2192139.25 acc: 0.29166334867477417
step: 465
train: loss: 1512702.875 acc: 0.30429065227508545  val: loss: 1811217.25 acc: 0.3320707082748413
step: 470
train: loss: 1640997.625 acc: 0.5882787108421326  val: loss: 2628220.0 acc: 0.6147364974021912
step: 475
train: loss: 591869.9375 acc: 0.6984331607818604  val: loss: 1540693.25 acc: 0.4983922839164734
step: 480
train: loss: 760876.0625 acc: 0.6615423560142517  val: loss: 1173828.625 acc: 0.6351436376571655
step: 485
train: loss: 1024309.5 acc: 0.5301979780197144  val: loss: 1782009.0 acc: 0.5956850051879883
step: 490
train: loss: 806536.0625 acc: 0.5923933982849121  val: loss: 847700.875 acc: 0.4709100127220154
step: 495
train: loss: 377595.65625 acc: 0.709386885

step: 865
train: loss: 731615.875 acc: 0.807420015335083  val: loss: 379060.71875 acc: 0.8010715842247009
step: 870
train: loss: 208550.5625 acc: 0.8559668064117432  val: loss: 781494.0 acc: 0.7622688412666321
step: 875
train: loss: 27859.34375 acc: 0.9757806658744812  val: loss: 2618828.5 acc: 0.6543935537338257
step: 880
train: loss: 16492.7421875 acc: 0.9869830012321472  val: loss: 1842820.625 acc: 0.7421854734420776
step: 885
train: loss: 121373.7734375 acc: 0.9113631248474121  val: loss: 940501.375 acc: 0.7492039203643799
step: 890
train: loss: 114505.421875 acc: 0.9274503588676453  val: loss: 601902.875 acc: 0.7746347188949585
step: 895
train: loss: 64437.71484375 acc: 0.9359728693962097  val: loss: 593616.625 acc: 0.776112973690033
step: 900
train: loss: 94868.6875 acc: 0.8979508280754089  val: loss: 4742634.5 acc: 0.6407012939453125
step: 905
train: loss: 59494.04296875 acc: 0.9455792307853699  val: loss: 1438736.0 acc: 0.7329958081245422
step: 910
train: loss: 236601.171875 ac

step: 1290
train: loss: 308512.5625 acc: 0.7043154835700989  val: loss: 313303.75 acc: 0.7467873096466064
step: 1295
train: loss: 286315.59375 acc: 0.8048158288002014  val: loss: 472865.84375 acc: 0.7379367351531982
step: 1300
train: loss: 740140.125 acc: 0.5879913568496704  val: loss: 517968.03125 acc: 0.7607003450393677
step: 1305
train: loss: 1545988.875 acc: 0.7584806680679321  val: loss: 658672.375 acc: 0.8132984042167664
step: 1310
train: loss: 688074.0 acc: 0.800528347492218  val: loss: 746221.625 acc: 0.7626786828041077
step: 1315
train: loss: 1318587.875 acc: 0.8849600553512573  val: loss: 1503696.125 acc: 0.7024616003036499
step: 1320
train: loss: 468838.46875 acc: 0.9440143704414368  val: loss: 720931.8125 acc: 0.8536074757575989
step: 1325
train: loss: 648637.75 acc: 0.8976967930793762  val: loss: 464276.28125 acc: 0.9285270571708679
step: 1330
train: loss: 328391.1875 acc: 0.928451418876648  val: loss: 2084926.375 acc: 0.7066618204116821
step: 1335
train: loss: 377682.2812

step: 1705
train: loss: 282709.1875 acc: 0.9684848785400391  val: loss: 767539.75 acc: 0.5928479433059692
step: 1710
train: loss: 291189.25 acc: 0.9795644879341125  val: loss: 1251889.0 acc: 0.805086076259613
step: 1715
train: loss: 392970.53125 acc: 0.9569621086120605  val: loss: 386425.375 acc: 0.8833233118057251
step: 1720
train: loss: 408685.46875 acc: 0.9487995505332947  val: loss: 399605.625 acc: 0.9060736298561096
step: 1725
train: loss: 120304.2890625 acc: 0.9515981078147888  val: loss: 747040.9375 acc: 0.8304352760314941
step: 1730
train: loss: 154174.796875 acc: 0.9368341565132141  val: loss: 439512.78125 acc: 0.9099471569061279
step: 1735
train: loss: 11351.208984375 acc: 0.9916791915893555  val: loss: 1206115.625 acc: 0.8778471350669861
step: 1740
train: loss: 15741.7587890625 acc: 0.9596683979034424  val: loss: 761146.6875 acc: 0.7538918256759644
step: 1745
train: loss: 12062.318359375 acc: 0.9921257495880127  val: loss: 351334.875 acc: 0.9132819771766663
step: 1750
train:

...

step: 48095
train: loss: 45477.13671875 acc: 0.9578261375427246  val: loss: 3354665.75 acc: 0.48081767559051514
step: 48100
train: loss: 509759.3125 acc: 0.6999608874320984  val: loss: 210291.046875 acc: 0.8132927417755127
step: 48105
train: loss: 262349.0625 acc: 0.8264302015304565  val: loss: 4749673.0 acc: 0.35008513927459717
step: 48110
train: loss: 119898.140625 acc: 0.9048857688903809  val: loss: 637030.1875 acc: 0.5715528726577759
step: 48115
train: loss: 143994.8125 acc: 0.9018825888633728  val: loss: 1501748.375 acc: 0.6423383951187134
step: 48120
train: loss: 1387228.25 acc: 0.742444634437561  val: loss: 1355218.375 acc: 0.6592402458190918
step: 48125
train: loss: 1059694.125 acc: 0.6617741584777832  val: loss: 2679061.75 acc: 0.7401236295700073
step: 48130
train: loss: 1074085.625 acc: 0.8859548568725586  val: loss: 1215196.125 acc: 0.37416213750839233
step: 48135
train: loss: 105486.3984375 acc: 0.9919976592063904  val: loss: 2450022.75 acc: 0.5178511142730713
step: 48140
t

step: 48570
train: loss: 9441.720703125 acc: 0.9555243253707886  val: loss: 672553.8125 acc: 0.9111570119857788
step: 48575
train: loss: 8297.8662109375 acc: 0.9816473126411438  val: loss: 1050287.375 acc: 0.7094697952270508
step: 48580
train: loss: 6592.23681640625 acc: 0.9668929576873779  val: loss: 2014001.875 acc: 0.26697057485580444
step: 48585
train: loss: 26394.431640625 acc: 0.965757429599762  val: loss: 1325213.75 acc: 0.6742355823516846
step: 48590
train: loss: 45786.796875 acc: 0.9684613347053528  val: loss: 653774.875 acc: 0.7487035989761353
step: 48595
train: loss: 35248.31640625 acc: 0.9842274188995361  val: loss: 397334.75 acc: 0.9035443067550659
step: 48600
train: loss: 16090.806640625 acc: 0.985318660736084  val: loss: 395262.3125 acc: 0.6309888362884521
step: 48605
train: loss: 55501.12890625 acc: 0.9775724411010742  val: loss: 215587.484375 acc: 0.9422951936721802
step: 48610
train: loss: 13822.32421875 acc: 0.9900500178337097  val: loss: 1120078.375 acc: 0.698146224

step: 49045
train: loss: 73590.6328125 acc: 0.9873536825180054  val: loss: 986979.875 acc: 0.8659175634384155
step: 49050
train: loss: 207905.75 acc: 0.9668378829956055  val: loss: 644888.9375 acc: 0.9145179986953735
step: 49055
train: loss: 426143.53125 acc: 0.9661587476730347  val: loss: 1896065.5 acc: -0.3816763162612915
step: 49060
train: loss: 625407.4375 acc: 0.9650444388389587  val: loss: 611719.75 acc: 0.9431138038635254
step: 49065
train: loss: 181149.578125 acc: 0.9637241959571838  val: loss: 1994467.125 acc: -0.08198809623718262
step: 49070
train: loss: 715732.1875 acc: 0.9528629779815674  val: loss: 2097939.0 acc: 0.6727315187454224
step: 49075
train: loss: 567407.0625 acc: 0.9775909781455994  val: loss: 669091.4375 acc: 0.9121337532997131
step: 49080
train: loss: 900122.6875 acc: 0.9707679152488708  val: loss: 719494.75 acc: 0.3829955458641052
step: 49085
train: loss: 812416.375 acc: 0.9524650573730469  val: loss: 1227023.125 acc: 0.8823916912078857
step: 49090
train: loss

step: 49510
train: loss: 506312.4375 acc: 0.7689269781112671  val: loss: 3224009.5 acc: 0.4991534948348999
step: 49515
train: loss: 117320.6640625 acc: 0.9033676981925964  val: loss: 2657293.75 acc: 0.434295117855072
step: 49520
train: loss: 116794.203125 acc: 0.913696825504303  val: loss: 2115218.25 acc: 0.6019682884216309
step: 49525
train: loss: 63823.72265625 acc: 0.9451017379760742  val: loss: 1537460.375 acc: 0.545351505279541
step: 49530
train: loss: 201350.203125 acc: 0.882332444190979  val: loss: 7017842.5 acc: 0.21188223361968994
step: 49535
train: loss: 301876.8125 acc: 0.843914270401001  val: loss: 1823425.75 acc: 0.5861752033233643
step: 49540
train: loss: 400736.21875 acc: 0.7869415879249573  val: loss: 848184.1875 acc: 0.6807624101638794
step: 49545
train: loss: 316036.4375 acc: 0.8264784812927246  val: loss: 1065501.25 acc: 0.5834380388259888
step: 49550
train: loss: 207507.046875 acc: 0.7788923978805542  val: loss: 1646373.125 acc: 0.5856924057006836
step: 49555
train:

step: 49985
train: loss: 66492.1171875 acc: 0.9952086210250854  val: loss: 755852.6875 acc: 0.857893705368042
step: 49990
train: loss: 209477.53125 acc: 0.9834766983985901  val: loss: 676006.0625 acc: 0.8963378667831421
step: 49995
train: loss: 175150.0625 acc: 0.9783071875572205  val: loss: 1280114.75 acc: 0.6899206638336182
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Run call was cancelled
